<a href="https://colab.research.google.com/github/VarunVejalla/Answer-Extraction-ASSIP/blob/master/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASSIP BERT Model


Import python resources

In [ ]:
import pandas as pd
import numpy as np
!pip install -e git+https://github.com/negedng/bert-embedding#egg=bert_embedding
!pip install transformers
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn import model_selection
#import matplotlib.pyplot as plt
#%matplotlib inline
import pickle # needed for opening entities
import json # needed for squad data
import gc

Read in data

In [ ]:
# read in csv
entitiesf1 = pd.read_csv("entities_train.csv")
testingf1 = pd.read_csv("entities_test.csv")

# loads in the squad datas
with open("train-v2.0.json") as file:
  squad_train = json.load(file)

with open("dev-v2.0.json") as file:
  squad_test = json.load(file)

# to test with the coreferences resolved, we used this command
# file = open("resolvedTexts", "rb")
# squad_train = pickle.load(file)

# file = open("resolvedTexts_test", "rb")
# squad_test = pickle.load(file)

# to cut time in the model, we created the embeddings beforehand for all passages, and then loaded those in.
# After loading them in and saving them to a list called doc_embeddings and testing_embeddings, we deleted the original variable names
# with garbage collector.

Import BERT Embeddings and Tokenizer

In [ ]:
from bert_embedding import BertEmbedding
from transformers import BertTokenizer
bert_embedding = BertEmbedding(max_seq_length=512)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Convert data into series (for regular text)

In [ ]:
list_of_para_per_doc = list()
for doc in range(len(squad_train["data"])):
  paragraphs = np.array([squad_train["data"][doc]["paragraphs"][para]["context"] for para in range(len(squad_train["data"][doc]["paragraphs"]))])
  paragraph_series = pd.Series(paragraphs)
  list_of_para_per_doc.append(paragraph_series)

list_of_para_per_doc_test = list()
for doc in range(len(squad_test["data"])):
  paragraphs = np.array([squad_test["data"][doc]["paragraphs"][para]["context"] for para in range(len(squad_test["data"][doc]["paragraphs"]))])
  paragraph_series = pd.Series(paragraphs)
  list_of_para_per_doc_test.append(paragraph_series)

Convert data into series (for resolved text)

In [ ]:
# list_of_para_per_doc = list()
# for doc in range(len(squad_train)):
#   paragraphs = np.array([squad_train[doc][para] for para in range(len(squad_train[doc]))])
#   paragraph_series = pd.Series(paragraphs)
#   list_of_para_per_doc.append(paragraph_series)

# list_of_para_per_doc_test = list()
# for doc in range(len(squad_test)):
#   paragraphs = np.array([squad_test[doc][para] for para in range(len(squad_test[doc]))])
#   paragraph_series = pd.Series(paragraphs)
#   list_of_para_per_doc_test.append(paragraph_series)

In [ ]:
del squad_train
del squad_test
gc.collect()
del gc.garbage[:]

Generate BERT embedding for every token of every sentence

In [ ]:
doc_embeddings = []
temp = 0
for doc_series in range(len(list_of_para_per_doc)):
  print(temp)
  embed = bert_embedding(list_of_para_per_doc[doc_series], filter_spec_tokens=False,)
  doc_embeddings.append(embed)
  print(len(doc_embeddings[doc_series][0][0]))
  temp+=1

In [ ]:
temp = 0
testing_embeddings = []
for test_series in range(len(list_of_para_per_doc_test)):
  print(temp)
  embed = bert_embedding(list_of_para_per_doc_test[test_series], filter_spec_tokens=False,)
  testing_embeddings.append(embed)
  print(len(testing_embeddings[test_series - 2][0][0]))
  temp+=1

Select the entity embeddings

In [ ]:
def split_entity(string_entity):
  temporary = ""
  if string_entity.isalnum():
    return string_entity
  if len(string_entity) > 0 and string_entity[0].isalnum() == False:
    return string_entity[0]
  for s in string_entity:
    if s.isalnum():
      temporary += s
    else:
      return temporary

In [ ]:
entities_embs = []
for entity_num in range(len(entitiesf1.entity)):
  doc_embs = entitiesf1.doc[entity_num]
  paragraph = entitiesf1.para[entity_num]
  if doc_embs < len(doc_embeddings):
    try:
      entity = entitiesf1.entity[entity_num]
      new_entity = entity.lower().strip('"')
      if ' ' in new_entity:
        new_entity = new_entity[:new_entity.index(' ')]
      if new_entity not in doc_embeddings[doc_embs][paragraph][0]:
        ne = "[CLS] " + new_entity + " [SEP]"
        tokenized_text = tokenizer.tokenize(ne) 
        new_entity_temporary = tokenized_text[0]
        if new_entity_temporary in doc_embeddings[doc_embs][paragraph][0]:
          new_entity = new_entity_temporary
        else:
          new_entity = split_entity(new_entity)
      entity_index = doc_embeddings[doc_embs][paragraph][0].index(new_entity)
      entities_embs.append(doc_embeddings[doc_embs][paragraph][1][entity_index])
    except ValueError:
      print(doc_embeddings[doc_embs][paragraph][0])
      print(entity)
      print(new_entity)
      print(new_entity_temporary)
      print(doc_embs)

Embedding for Test

In [ ]:
testing_embs = []
for test_entity_num in range(len(testingf1.entity)):
  test_embs = testingf1.doc[test_entity_num]
  paragraph = testingf1.para[test_entity_num]
  if test_embs < len(testing_embeddings):
    try:
      entity = testingf1.entity[test_entity_num]
      new_entity = entity.lower().strip('"')
      if ' ' in new_entity:
        new_entity = new_entity[:new_entity.index(' ')]
      if new_entity not in testing_embeddings[test_embs][paragraph][0]:
        ne = "[CLS] " + new_entity + " [SEP]"
        tokenized_text = tokenizer.tokenize(ne) 
        new_entity_temporary = tokenized_text[0]
        if new_entity_temporary in testing_embeddings[test_embs][paragraph][0]:
          new_entity = new_entity_temporary
        else:
          new_entity = split_entity(new_entity)
      entity_index = testing_embeddings[test_embs][paragraph][0].index(new_entity)
      testing_embs.append(testing_embeddings[test_embs][paragraph][1][entity_index])
    except ValueError:
      print(testing_embeddings[test_embs][paragraph][0])
      print(testingf1.entity[test_entity_num])
      print(new_entity)
      print(test_embs)

In [ ]:
del doc_embeddings
del testing_embeddings
gc.collect()
del gc.garbage[:]

In [ ]:
entities_embs = np.array(entities_embs)
entities_embs.shape

In [ ]:
testing_embs = np.array(testing_embs)
testing_embs.shape

PCA projection to 2D

In [ ]:
entities_pca = PCA(n_components=2).fit_transform(entities_embs)
entities_pca.shape

Evaluate a k-NN (k-Nearest Neighbour Classifier) model with LOOCV (Leave One Out Cross-Validation)

In [ ]:
loocv = model_selection.LeaveOneOut()
model = KNeighborsClassifier(n_neighbors=8)
results = model_selection.cross_val_score(model, entities_embs, entitiesf1.Type, cv=loocv)
print("Accuracy: %.3f%% (STDev %.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Train the model

In [ ]:
trainedModel = model.fit(entities_embs, entitiesf1.Type)

Test the Model

In [ ]:
wrong = 0
total = 0
predictions = trainedModel.predict(testing_embs)
for i in range(len(testingf1.Type)):
  if predictions[i] != testingf1.Type[i]:
    print(i)
    wrong += 1
  total += 1

print(str(total - wrong) + " Correct out of " + str(total))